In [1]:
from __future__ import print_function

import datetime
import os.path
import json

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']


def main():
    """
    Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('calendar', 'v3', credentials=creds)

        # Call the Calendar API
        # Get start and end dates for the query
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time

        next_week = (datetime.datetime.today() + datetime.timedelta(days = 7)).isoformat()[:10]+"T23:59:59.99Z"
        end_date = next_week[:10]

        events_result = service.events().list(
            calendarId='bozemanfellowship@gmail.com', 
            timeMin=now,
            timeMax = next_week, 
#             maxResults=n, # in case you'd rather just get N events
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        
        if not events:
            print('No upcoming events found.')
            return
            
        meetings = []
        for e in events:
            if 'location' in e:
                loc = e['location']
            else:
                loc = None
            meetings.append(
                {
                    'title': e['summary'],
                    'location': loc,
                    'start_time': datetime.datetime.strftime(datetime.datetime.strptime(e['start']['dateTime'], '%Y-%m-%dT%H:%M:%S%z'), "%Y-%m-%d %I:%M %p") 
#                     'end_time': datetime.datetime.strftime(datetime.datetime.strptime(e['end']['dateTime'], '%Y-%m-%dT%H:%M:%S%z'), "%Y-%m-%d %I:%M %p"), 
                }
            )
        
        # Print the list of meeting information
#         print('\n\n', meetings)

        with open(f'events_ending_{end_date}.json', 'w', encoding='utf-8') as f:
            json.dump(meetings, f, ensure_ascii=False, indent=4)
        
    except HttpError as error:
        print('An error occurred: %s' % error)


if __name__ == '__main__':
    main()

In [ ]:
# two files
# 1. daily: title, location, start time
# 2. Weekly: title, start time